In [1]:
import sys
sys.path.append("..")

In [2]:
import matplotlib.pyplot as plt
from utils_reboot.datasets import Dataset
import os
import numpy as np
import pickle
import time
from matplotlib.ticker import AutoLocator, ScalarFormatter

In [3]:
os.chdir("../")
data_path=os.getcwd()+"/data/syn/"

In [4]:
def get_vals(model, 
            dataset_names,
            type='predict'):
    
    with open(os.getcwd() + "/utils_reboot/time_scaling_test.pickle", "rb") as file:
        dict_time = pickle.load(file)

    val_times=[]
    for d_name in dataset_names:
        time=np.array(dict_time[type][model][d_name])
        val_times.append(time)

    median_val_times=[np.percentile(x,50) for x in val_times]
    five_val_times=[np.percentile(x,5) for x in val_times]
    ninefive_val_times=[np.percentile(x,95) for x in val_times]

    return median_val_times,five_val_times,ninefive_val_times

In [37]:
model_names=['EIF+','EIF','sklearn_IF','DIF','AnomalyAutoencoder']
importance_names=['EXIFFI+','EXIFFI','DIFFI','RandomForest']
dataset_names=['Xaxis_100_6','Xaxis_250_6','Xaxis_500_6',
              'Xaxis','Xaxis_2500_6','Xaxis_5000_6',
              'Xaxis_10000_6','Xaxis_25000_6','Xaxis_50000_6','Xaxis_100000_6']
dataset_features_names=['Xaxis_5000_6','Xaxis_5000_12','Xaxis_5000_32','Xaxis_5000_64','Xaxis_5000_128']
plot_path = os.getcwd()+"/experiments/results/time_scaling_plots"


In [35]:
def plot_time_scaling(model_names,
                        dataset_names,
                        data_path,
                        type='predict',
                        plot_type='samples',
                        plot_path=plot_path,
                        show_plot=True,
                        save_plot=True):

    assert type in ['predict','fit','importances'], "Type not valid. Accepted values: ['predict','fit','importances'] "
    assert plot_type in ['samples','features'], "Plot Type not valid. Accepted values: ['samples','features']"
    
    datasets=[Dataset(name,path=data_path) for name in dataset_names]

    if plot_type == "samples":
        sample_sizes=[data.shape[0] for data in datasets]
    elif plot_type == "features":
        sample_sizes=[data.shape[1] for data in datasets]

    fig, ax = plt.subplots()
    plt.style.use('default')
    plt.rcParams['axes.facecolor'] = '#F2F2F2'
    plt.grid(alpha = 0.7)
    colors = ["tab:red","tab:blue","tab:orange","tab:green","tab:blue"]

    maxs=[]
    mins=[]
    for i,model in enumerate(model_names):
        median_times,five_times,ninefive_times=get_vals(model,dataset_names,type=type)
        maxs.append(np.max(median_times))
        mins.append(np.min(median_times))

        ax.plot(sample_sizes,median_times,alpha=0.85,c=colors[i],marker="o",label=model)
        ax.fill_between(sample_sizes,five_times,ninefive_times,alpha=0.1,color=colors[i])
    
    if plot_type == "samples":
        ax.set_yscale('log')
        ax.set_xscale("log")
        ax.yaxis.set_major_locator(AutoLocator())
        ax.yaxis.set_major_formatter(ScalarFormatter())
        ax.minorticks_off()
        ax.set_xticks(sample_sizes,sample_sizes,rotation=45,fontsize = 12)
        ax.set_yticks([1,10,25,100,250],fontsize = 14)
        
    ax.set_xlabel('Sample Size',fontsize = 20)
    ax.set_ylabel(f'{type} Time (s)',fontsize = 20)
    #plt.ylim(np.min(mins)-0.2*np.min(mins),np.max(maxs)+0.2*np.max(maxs))

    
    ax.legend()
    ax.grid(visible=True, alpha=0.5, which='major', color='gray', linestyle='-')
    
    t = time.localtime()
    current_time = time.strftime("%d-%m-%Y_%H-%M-%S", t)

    if save_plot:
        plt.savefig(f'{plot_path}/{current_time}_time_scaling_plot_{plot_type}_{type}.pdf',bbox_inches='tight')

    if show_plot:
        plt.show()
    
    return fig,ax


In [38]:
plot_time_scaling(importance_names,dataset_features_names,data_path,type="importances",plot_type="features")

Exception: The dataset name is not valid

UsageError: Line magic function `%/usr/local/bin/python3` not found.
